#### Importing Libraries

In [15]:
import numpy as np
import pandas as pd
import json
import requests
import googlemaps
from googlemaps import places
from datetime import datetime
import time

#### Getting the Dataframe with mesh points

In [28]:
dub = pd.read_csv('Dublin_mesh_76_no_index.csv') 

#### Function to reduce the dataframe into a few rows for testing purposes

In [29]:
# Generates a new dataframe with 5 random rows
def random_rows(df, nrows):
    import random
    df_random = pd.DataFrame()
    rnd_lst = [random.randint(0, len(df)) for iter in range(nrows)]
    for num in rnd_lst:
        df_random = df_random.append(dub.iloc[num, :])
    df_random = df_random.reset_index(drop=True)
    return df_random

In [214]:
dm = pd.DataFrame()
dm = random_rows(dub, 1)

In [242]:
dm = dub

In [243]:
dm.shape

(76, 5)

### Google Places API

In [14]:
# @hidden_cell
gmaps = googlemaps.Client(key='AIzaxxxxxxxxxxxxxxxxxxxxxxxxxxPTo')

### <a id="data_ac">Data Acquisition</a>

### Creating new dataframe with venues using Google Places API
 Venues Category is set to filter the search. Distance between Mesh points is 500m. Setting radius to 365m.

#### Function to get nearby places using Google Places API

In [245]:
def getNearbyVenues(names, latitudes, longitudes, category, radius=365):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        space = '     '
        print(name +'{}'.format(10*space), end="\r", flush=True)
        print(name)
        token = None
        location = '{}, {}'.format(lat, lng)
        print (location)
        query = places.places_nearby(gmaps, location, radius=radius, keyword=category, page_token=token )
        print ('Raw first query. keys {}'.format(query.keys()))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['geometry']['location']['lat'], 
            v['geometry']['location']['lng'],
            v['id'],
            v.get("price_level", 0),
            v.get("rating", 0),
            v.get("user_ratings_total", 0),
            v['types']) for v in query['results']])
        print('Page 1 processed.')
        i=2    
        while "next_page_token" in query:
                time.sleep(1)
                if i==2:
                   print ('getting page {} from location {}. 2 secs...'.format(str(i), location))
                else:   
                   print ('previous token for page {} was: {}...'.format(str(i-1), token[:20]))
                token = query.get('next_page_token')
                print ('token using for page {} from location {}. token: {}...'.format(str(i), location, token[:20]))
                time.sleep(1)
                query = places.places_nearby(gmaps, location, radius=radius, keyword=category, page_token=token )
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['name'], 
                    v['geometry']['location']['lat'], 
                    v['geometry']['location']['lng'],
                    v['id'],
                    v.get("price_level", 0),
                    v.get("rating", 0),
                    v.get("user_ratings_total", 0),
                    v['types']) for v in query['results']])
                print('Page {} processed.'.format (str(i)))
                i+=1
                time.sleep(1)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue id',
                  'Venue price level',           
                  'Venue Rating',           
                  'Total user ratings',                 
                  'Venue Category']

    return(nearby_venues)

In [ ]:
dublin_gmap_venues = getNearbyVenues(names = dm['PostCode'],
                                   latitudes = dm['Latitude'],
                                   longitudes = dm['Longitude'],
                                   category = 'food'
                                   #category = None
                                  )

We requested first information without filtering using a keyword, and after using 'food' as keyword

In [248]:
dublin_gmap_venues.to_csv('CSVs/Gmap_Dublin_updated_keyword_food.csv', index=False)

In [249]:
dub_raw = dublin_gmap_venues.copy()

In [247]:
dublin_gmap_venues.shape

(2168, 11)

In [202]:
dub_raw.columns

Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue id',
       'Venue price level', 'Venue Rating', 'Total user ratings',
       'Venue Category'],
      dtype='object')

In [250]:
dub_clean = dub_raw.drop_duplicates(['Venue Latitude', 'Venue Longitude', 'Venue id'], keep='first')
dub_clean.shape

(669, 11)

dub_food = dub_clean[dub_clean['Venue Category'].astype(str).str.contains('food')]

dub_food.shape

#print('There are {} uniques categories.'.format(len(dub_food['Venue Category'].unique())))

In [253]:
import folium
dub_latitude = '53.341785'
dub_longitude = '-6.265288'
# create map of Dublin using latitude and longitude values
map_dublin = folium.Map(location=[dub_latitude, dub_longitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(dub_clean['Venue Latitude'], dub_clean['Venue Longitude'], dub_clean['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dublin)

map_dublin

In [195]:
print(dub_1.shape[0])
print(dub_2.shape[0])
print(dub_3.shape[0])
print(dub_4.shape[0])

60
60
60
60


In [211]:
dub_3

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue id,Venue price level,Venue Rating,Total user ratings,Venue Category
0,Dublin 3,53.356207,-6.242594,Dublin,53.349805,-6.260310,71ccb3b82cc1f534c1da9998f8610d85a6d7c598,0,0.0,0,"[locality, political]"
1,Dublin 3,53.356207,-6.242594,Jurys Inn Christchurch Dublin,53.342832,-6.270520,2ccbdecd28694cc8b74bfe56279db7fb52187101,0,4.2,1320,"[lodging, restaurant, food, point_of_interest,..."
2,Dublin 3,53.356207,-6.242594,Kinlay House Dublin,53.343726,-6.269898,7511e298b05c19078d13d98bb64909a1e62549f7,0,3.9,1110,"[lodging, point_of_interest, establishment]"
3,Dublin 3,53.356207,-6.242594,"Radisson Blu Royal Hotel, Dublin",53.340853,-6.268325,5245b21949a3fad5785f03ff1427446a7339d412,0,4.3,1363,"[lodging, point_of_interest, establishment]"
4,Dublin 3,53.356207,-6.242594,Harding Hotel,53.343800,-6.269900,ecb52facb48ba5c14e3048bc8323a4854a90a76f,0,4.4,404,"[lodging, point_of_interest, establishment]"
5,Dublin 3,53.356207,-6.242594,Handels Hotel by thekeycollections,53.344138,-6.269632,acf31088919ecd6e9287f8cf6d567a8951b50aa3,0,3.8,300,"[lodging, point_of_interest, establishment]"
6,Dublin 3,53.356207,-6.242594,The Four Courts Hostel,53.344986,-6.273895,03a65798dd143cf29a42cae5cb5a4b5623cdac2c,0,3.9,685,"[lodging, point_of_interest, establishment]"
7,Dublin 3,53.356207,-6.242594,Christchurch Guinness Ground Floor,53.342680,-6.275572,9b314dae3d561a66bbf4a982e34d06c96da32eb5,0,0.0,0,"[lodging, point_of_interest, establishment]"
8,Dublin 3,53.356207,-6.242594,The Viking Lodge Hotel,53.341507,-6.274602,84077a8d4dfb8aa660cf7f06e7efa577f26b19d7,0,4.0,5,"[lodging, point_of_interest, establishment]"
9,Dublin 3,53.356207,-6.242594,"Staycity Aparthotels - Christchurch, Dublin",53.343095,-6.273901,d51ef06c53eb5124caf83055ec99133f9f50f53b,0,4.0,299,"[lodging, point_of_interest, establishment]"


In [240]:
dub_3t = dublin_gmap_venues.copy()

In [241]:
dub_3t

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue id,Venue price level,Venue Rating,Total user ratings,Venue Category
0,Dublin 2,53.336978,-6.257723,Communist Party of Ireland,53.345027,-6.266496,f2900cde6aced8c6a2579e9506a06d0c2c250b0b,0,3.5,8,"[point_of_interest, establishment]"
1,Dublin 2,53.336978,-6.257723,The Labour Party,53.339285,-6.266151,ba55a674cfad87d7fdcf1272930948f0b47b2b4b,0,3.3,3,"[point_of_interest, establishment]"
2,Dublin 2,53.336978,-6.257723,Dublin Castle,53.342886,-6.267428,61f3f973668a4012269ca6e9e11957f141146b7e,0,4.2,20362,"[tourist_attraction, point_of_interest, establ..."


### <center>[NEXT CHAPTER](./3.TA_Web_Scraping.ipynb#data_ac)</center>

#### <center> [Table of Contents](./0.Table_of_Contents_Code.ipynb) </center>